<a href="https://colab.research.google.com/github/OlhaYastrebova/goit-ds-hw/blob/main/Hw10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise
import surprise
from surprise import Dataset, SVD, SVDpp, NMF
from surprise.model_selection import GridSearchCV, cross_validate

# Завантаження даних
data = Dataset.load_builtin('ml-100k')
data


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357283 sha256=38aeba232da1ec72f9d93a670a0c517a8fb1f8d359061f483bd0916cc62ba501
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise
Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
# Набір параметрів для підбору
param_grid_svd = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30, 40],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.1]
}

# Пошук найкращих параметрів для кожного алгоритму
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse'], cv=5)
gs_svd.fit(data)


In [ ]:
# Виведення результатів
print("Best SVD score: ", gs_svd.best_score['rmse'])
print("Best SVD parameters: ", gs_svd.best_params['rmse'])

Best SVD score:  0.9169508790300263
Best SVD parameters:  {'n_factors': 150, 'n_epochs': 40, 'lr_all': 0.005, 'reg_all': 0.1}


In [ ]:
param_grid_svdpp = {
    'n_factors': [20],
    'n_epochs': [20],
    'lr_all': [0.002],
    'reg_all': [0.02]
}

gs_svdpp = GridSearchCV(SVDpp, param_grid_svdpp, measures=['rmse'], cv=5)
gs_svdpp.fit(data)

print("Best SVD++ score: ", gs_svdpp.best_score['rmse'])
print("Best SVD++ parameters: ", gs_svdpp.best_params['rmse'])

Best SVD++ score:  0.9406952920172081
Best SVD++ parameters:  {'n_factors': 20, 'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.02}


In [ ]:
param_grid_nmf = {
    'n_factors': [50],
    'n_epochs': [50],
    'reg_pu': [0.1],
    'reg_qi': [0.1]
}

gs_nmf = GridSearchCV(NMF, param_grid_nmf, measures=['rmse'], cv=5)
gs_nmf.fit(data)

print("Best NMF score: ", gs_nmf.best_score['rmse'])
print("Best NMF parameters: ", gs_nmf.best_params['rmse'])


Best NMF score:  0.9403116242041067
Best NMF parameters:  {'n_factors': 50, 'n_epochs': 50, 'reg_pu': 0.1, 'reg_qi': 0.1}


In [ ]:
param_grid_nmf = {
    'n_factors': [100],
    'n_epochs': [100],
    'reg_pu': [0.1],
    'reg_qi': [0.1]
}

gs_nmf = GridSearchCV(NMF, param_grid_nmf, measures=['rmse'], cv=5)
gs_nmf.fit(data)

print("Best NMF score: ", gs_nmf.best_score['rmse'])
print("Best NMF parameters: ", gs_nmf.best_params['rmse'])


Best NMF score:  0.9200549387085248
Best NMF parameters:  {'n_factors': 100, 'n_epochs': 100, 'reg_pu': 0.1, 'reg_qi': 0.1}


In [ ]:
# Оцінка моделей на основі найкращих параметрів
best_svd = gs_svd.best_estimator['rmse']
best_svdpp_params = gs_svdpp.best_params['rmse']
best_nmf_params = gs_nmf.best_params['rmse']

# Створення моделей з найкращими параметрами
best_svd = gs_svd.best_estimator['rmse']
best_svdpp = SVDpp(**best_svdpp_params)
best_nmf = NMF(**best_nmf_params)

# Список моделей для оцінки
models = [("SVD", best_svd),("SVD++", best_svdpp), ("NMF", best_nmf)]

# Оцінка кожної моделі за допомогою крос-валідації
for name, model in models:
    results = cross_validate(model, data, measures=['rmse'], cv=5, verbose=False)
    print(f"{name} mean RMSE: {results['test_rmse'].mean():.4f}")

SVD mean RMSE: 0.9191
SVD++ mean RMSE: 0.9412
NMF mean RMSE: 0.9206


Можна зробити висновок, що алгоритм SVD показує найкращу якість прогнозування з найменшим середнім значенням RMSE, хоча різниця між SVD і NMF є мінімальною. SVD++ демонструє трохи гірші результати у порівнянні з іншими методами.

Таким чином, для даного завдання оптимальним вибором буде використання SVD.